In [ ]:
# Random Forest
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score
from sklearn.preprocessing import StandardScaler

# Load your data
data = pd.read_excel('ml eyes closed dataset/hc_vs_mci_closed.xlsx')  # Update with your actual file name

# Define brain regions combinations based on the user's request
combinations = {
    

        'Frontal': ['Fp1', 'F3', 'F4', 'Fz', 'F7', 'F8'],
        'Occipital': ['O1', 'O2'],
        'Central': ['C3', 'C4', 'Cz'],
        'Temporal': ['T3', 'T4', 'T5', 'T6'],
        'Parietal': ['P3', 'P4', 'Pz'],

        'Right Hemisphere': ['Fp2', 'F4', 'F8', 'C4', 'T4', 'P4', 'T6', 'O2'],
        'Left Hemisphere': ['Fp1', 'F3', 'F7', 'C3', 'T3', 'P3', 'T5', 'O1'],
        'Combined Hemisphere': ['Fp2', 'F4', 'F8', 'C4', 'T4', 'P4', 'T6', 'O2', 'Fp1', 'F3', 'F7', 'C3', 'T3', 'P3', 'T5', 'O1'],

        'Frontal, Central, Parietal': ['Fp1', 'F3', 'F4', 'Fz', 'F7', 'F8', 'C3', 'C4', 'Cz', 'P3', 'P4', 'Pz'],
        'Frontal, Central, Occipital': ['Fp1', 'F3', 'F4', 'Fz', 'F7', 'F8', 'C3', 'C4', 'Cz', 'O1', 'O2'],
        'Frontal, Parietal, Temporal': ['Fp1', 'F3', 'F4', 'Fz', 'F7', 'F8', 'P3', 'P4', 'Pz', 'T3', 'T4', 'T5', 'T6'],
        'Frontal, Parietal, Occipital': ['Fp1', 'F3', 'F4', 'Fz', 'F7', 'F8', 'P3', 'P4', 'Pz', 'O1', 'O2'],
        'Frontal, Temporal, Occipital': ['Fp1', 'F3', 'F4', 'Fz', 'F7', 'F8', 'T3', 'T4', 'T5', 'T6', 'O1', 'O2'],
        'Central, Parietal, Temporal': ['C3', 'C4', 'Cz', 'P3', 'P4', 'Pz', 'T3', 'T4', 'T5', 'T6'],
        'Central, Parietal, Occipital': ['C3', 'C4', 'Cz', 'P3', 'P4', 'Pz', 'O1', 'O2'],
        'Central, Temporal, Occipital': ['C3', 'C4', 'Cz', 'T3', 'T4', 'T5', 'T6', 'O1', 'O2'],
        'Parietal, Temporal, Occipital': ['P3', 'P4', 'Pz', 'T3', 'T4', 'T5', 'T6', 'O1', 'O2'],

        'Frontal, Central, Parietal, Occipital': ['Fp1', 'F3', 'F4', 'Fz', 'F7', 'F8', 'C3', 'C4', 'Cz', 'P3', 'P4', 'Pz', 'O1', 'O2'],
        'Frontal, Temporal, Parietal, Occipital': ['Fp1', 'F3', 'F4', 'Fz', 'F7', 'F8', 'T3', 'T4', 'T5', 'T6', 'P3', 'P4', 'Pz', 'O1', 'O2'],
        'Frontal, Temporal, Central, Occipital': ['Fp1', 'F3', 'F4', 'Fz', 'F7', 'F8', 'C3', 'C4', 'Cz', 'T3', 'T4', 'T5', 'T6', 'O1', 'O2'],
        'Temporal, Central, Parietal, Occipital': ['T3', 'T4', 'T5', 'T6', 'C3', 'C4', 'Cz', 'P3', 'P4', 'Pz', 'O1', 'O2'],
}
   
# Initialize the StandardScaler
scaler = StandardScaler()

# Prepare the results storage for Random Forest
results_rf = []

# Prepare the model training and evaluation for each combination
for name, regions in combinations.items():
    # Subset the data
    X = data[regions]
    y = data['Dx']
    
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # Standardize the features
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Initialize the Random Forest model
    rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
    
    # Train the Random Forest model
    rf_model.fit(X_train_scaled, y_train)
    
    # Predictions on the test set
    y_pred = rf_model.predict(X_test_scaled)
    y_pred_proba = rf_model.predict_proba(X_test_scaled)[:, 1]  # probabilities for AUC
    
    # Compute the evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    report = classification_report(y_test, y_pred, output_dict=True)
    auc = roc_auc_score(y_test, y_pred_proba)  # AUC computation
    
    # Store the results
    results_rf.append({
        'Combination': name,
        'Accuracy': accuracy,
        'Sensitivity': report['1']['recall'],
        'Specificity': report['0']['recall'],
        'PPV': report['1']['precision'],
        'NPV': report['0']['precision'],
        'AUC': auc,
        'Confusion Matrix': cm
    })

# Convert the results to a DataFrame
results_rf_df = pd.DataFrame(results_rf)
display_columns = ['Combination', 'Accuracy', 'Sensitivity', 'Specificity', 'PPV', 'NPV', 'AUC', 'Confusion Matrix']

# Save the results to an Excel file
results_rf_df.to_excel('hc_vs_mci_eyes_closed_results.xlsx', index=False)

# Optionally display the DataFrame (if in a Jupyter environment)
results_rf_df[display_columns]
